In [1]:
#Step 1: import pandas, streamlit and geopy libraries
import streamlit as st
import pandas as pd
from geopy.distance import geodesic

In [2]:
#Step 2: Name the Project
st.set_page_config(page_title="My Travel App")

In [3]:
#Step 3: Get Dataset
@st.cache_data
def get_dataset():
    url = "http://island.ricerca.di.unimi.it/~alfio/shared/worldcities.xlsx"
    return pd.read_excel(url)

#Set dataset
world_cities_dataset = get_dataset()

2023-11-20 02:28:44.991 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-11-20 02:28:45.281 
  command:

    streamlit run /Users/azatbekovna/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-11-20 02:28:45.283 No runtime found, using MemoryCacheStorageManager


In [4]:
#Step 4: Return dataset
world_cities_dataset

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140
...,...,...,...,...,...,...,...,...,...,...,...
26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709
26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491
26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438
26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159


In [5]:
#Step 4: Introduce helper functions

#get city
def get_city_id(city, cities_map):
    return cities_map[city]

#calculates ditance
def calculate_distance(city1_id, city2_id):
    coords1 = (world_cities_dataset.loc[world_cities_dataset['id'] == city1_id, 'lat'].values[0],
                       world_cities_dataset.loc[world_cities_dataset['id'] == city1_id, 'lng'].values[0])   #ex:coordinates of seould
    coords2 = (world_cities_dataset.loc[world_cities_dataset['id'] == city2_id, 'lat'].values[0],
                       world_cities_dataset.loc[world_cities_dataset['id'] == city2_id, 'lng'].values[0]) #ex:coordinates of tokyo
    return geodesic(coords1, coords2).kilometers  #ex:1000

#calculate time cost
def calculate_travelling_cost(city1_id, city2_id, i):
    weight = 2 * (i + 1)  # 2 hours for the nearest, 4 for the second, 8 for the third, etc. #2
    city_1_country = world_cities_dataset.loc[world_cities_dataset['id'] == city1_id, 'country'].values[0]
    city_2_country = world_cities_dataset.loc[world_cities_dataset['id'] == city2_id, 'country'].values[0]
    city_2_population = world_cities_dataset.loc[world_cities_dataset['id'] == city2_id, 'population'].values[0]
    if city_1_country != city_2_country:
        weight += 2  # Additional 2 hours if the destination city is in another country
    if city_2_population > 200000:
        weight += 2  # Additional 2 hours if the destination city has more than 200,000 inhabitants
    return weight

#longitude of defined city(since we know that lng increases when we go east)
def get_lng_city(city):
    return lng

In [1]:
#App starts here

st.title("Travel App")
st.write("This is a travel app that will help you to identify if you would be able to travel the world from the selected city in 80 days")

cities_map = {f"{city}": id for id, city in
                        zip(world_cities_dataset['id'], world_cities_dataset['city'])}
    #ex: {"London": "458", "Tokyo: "577}

start_city = st.selectbox('Select your starting location', list(cities_map)) #Seoul


selected_cities = st.multiselect('Select cities you want to visit', list(cities_map)) #{Tokyo, Jakarta, Manila}
st.write(f"You selected {selected_cities}")